In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
from tqdm import tqdm
import re, gensim
from googletrans import Translator

In [2]:
def sent_to_words(sentences):
  for sentence in sentences:
    sentence = re.sub(r'https?://\S+|www\.\S+', '', sentence)
    sentence = ''.join([char for char in sentence if char.isascii()])
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
  
def list_of_sentence(text):
    sentences = sent_tokenize(text.lower())
    los = list(sent_to_words(sentences))
    return los

In [3]:
def write_list_of_lists_to_text(file_path, data, delimiter='\t'):
    with open(file_path, 'w') as file:
        for sublist in data:
            line = delimiter.join(sublist)
            file.write(line + '\n')

def import_text_to_list_of_lists(file_path, delimiter='\t'):
    result = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            sublist = line.split(delimiter)
            result.append(sublist)
    return result

In [4]:
def page_contents(link, div_class = 'elementor-widget-container',tag='p'):
    driver = webdriver.Safari()
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    div_elements = soup.find_all('div', class_=div_class)

    paragraphs = []
    for div in div_elements:
        div_paragraphs = div.find_all(tag)
        for paragraph in div_paragraphs:
            paragraphs.append(paragraph.text.replace('\xa0', ''))

    driver.quit()
    return paragraphs

In [5]:
def translate_text(text, target_language):
    translator = Translator()
    detected_lang = "th"
    translated_text = translator.translate(text, src=detected_lang, dest=target_language)
    return translated_text.text

In [ ]:
#Scraping links on news page with click load more button

driver = webdriver.Safari()
driver.get('https://scbtechx.io/news/')
wait_duration = 5

while True:
    try:
        # Find the button element by its class
        load_more_button = driver.find_element(By.CLASS_NAME, "wpr-load-more-btn")
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)

    except ElementNotInteractableException:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')
section = soup.find("section", class_="wpr-grid elementor-clearfix grid-images-loaded")
a_tags = section.find_all("a")

keep_links = [link.get('href') for link in a_tags]
news_lisks = [link for link in set(keep_links) if link.startswith('https://scbtechx.io/') and 'https://scbtechx.io/category/' not in link]

driver.quit()

In [ ]:
links = ['https://scbtechx.io/privacy-notice/customers/',
         'https://scbtechx.io/privacy-notice/non-customers/',
         'https://scbtechx.io/data-platform/',
         'https://scbtechx.io/ekyc/',
         'https://scbtechx.io/ekyc-innovestx/',
         'https://scbtechx.io/services-products/',
         'https://scbtechx.io/terms-of-use/']

links += news_lisks

all_contents = []

for link in tqdm(links):
    tag = 'p'
    if link == 'https://scbtechx.io/services-products/':
        tag = 'h2'
    all_contents.append(page_contents(link,tag=tag))
    sleep(10)

In [ ]:
all_corpus = [list_of_sentence(" ".join(c)) for c in all_contents]
los_all_corpus = [list_of_sentence for corpus in all_corpus for list_of_sentence in corpus]

In [ ]:
file_path = 'src/Corpus_TechX_new.txt'
write_list_of_lists_to_text(file_path, los_all_corpus, delimiter='\t')
# techx_los = import_text_to_list_of_lists(file_path, delimiter='\t')

In [ ]:
# # Check word freq
# from collections import defaultdict

# word_freq = defaultdict(int)
# for sublist in techx_los:
#     for word in sublist:
#         word_freq[word] += 1
# word_freq = dict(word_freq)

PointX

In [ ]:
#Scping links on promotion page

driver = webdriver.Safari()
driver.get('https://www.pointx.scb/promotions/')

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(driver.page_source, "html.parser")


div = soup.find("div", class_="special-promotion-carousel-container carousel-share")
a_tags = div.find_all("a")

links = [a.get("href") for a in a_tags]
links = list(set(links))    # drop duplicate

driver.quit()

In [ ]:
def pointx_page_contents(link):
    driver = webdriver.Safari()
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    p_tags = soup.find_all("p")
    p_texts = [re.sub(r"[\n\t]"," ",tag.text )for tag in p_tags if tag.text.strip()]
    
    li_tags = soup.find_all("li")
    li_texts = [re.sub(r"[\n\t]"," ",tag.text ) for tag in li_tags if tag.text.strip()]

    driver.quit()
    return p_texts + li_texts

In [ ]:
pages_cons = []

for link in tqdm(links):
    pages_cons.append(pointx_page_contents(link))
    sleep(10)

In [ ]:
all_corpus = []
for content in tqdm(pages_cons):
    los_en = [translate_text(sentence,'en') for sentence in content]
    all_corpus.append(list_of_sentence(" ".join(los_en)))

los_all_corpus = [list_of_sentence for corpus in all_corpus for list_of_sentence in corpus]

In [ ]:
file_path = 'src/PointX_corpus.txt'
write_list_of_lists_to_text(file_path, los_all_corpus, delimiter='\t')
# pointx_los = import_text_to_list_of_lists(file_path, delimiter='\t')

SCB

In [ ]:
Title = "Business Maker"

In [ ]:
driver = webdriver.Safari()
driver.get('https://www.scb.co.th/en/personal-banking/stories.html')

wait_duration = 5
rounds = True
tip4u_Title_button = driver.find_element(By.XPATH, f'//a[@class="ga_menu_section" and @title="{Title}"]')
tip4u_Title_button.click()

while (rounds):
    try:
        # Find the button element by its class
        load_more_button = driver.find_element(By.XPATH, '//a[@class="btn-default mar-top ga_see_all_button" and @title="Load more"]')
        # driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)
        # rounds -= 1
    except :
        break

div_element = driver.find_element(By.CSS_SELECTOR, "div.row.relative.clearfix")

link_elements = div_element.find_elements(By.TAG_NAME, "a")

links = [link.get_attribute("href") for link in link_elements]
links = list(set(links))    # drop duplicate

driver.quit()

In [ ]:
len(links)

In [ ]:
pages_cons = []
for link in tqdm(links):
    pages_cons.append(page_contents(link,div_class="content"))
    sleep(10)

In [ ]:
all_corpus = []
for c in pages_cons:
    all_corpus.append(list_of_sentence(" ".join(c)))
    
los_all_corpus = [list_of_sentence for corpus in all_corpus for list_of_sentence in corpus]

In [ ]:
file_path = 'src/SCB_corpus_businessmaker.txt'
write_list_of_lists_to_text(file_path, los_all_corpus, delimiter='\t')
# scb_tip4u_los = import_text_to_list_of_lists(file_path, delimiter='\t')

InnoverstX

In [13]:
def get_news(page_link):
    driver = webdriver.Safari()
    driver.get(page_link)
    div_elements = driver.find_elements(By.CLASS_NAME, 'blog_card_body_lower')
    for div in div_elements:
        a_tag = div.find_element(By.TAG_NAME, 'a')
        yield a_tag.text, a_tag.get_attribute('href')
        # print(a_tag.text, a_tag.get_attribute('href'))
    sleep(5)
    driver.quit()

In [14]:
driver = webdriver.Safari()
driver.get('https://www.innovestx.co.th/wealth-playlist')
wait_duration = 5
rounds = True

while (rounds):
    try:
        # Find the button element by its class
        load_more_button = driver.find_element(By.XPATH, '//a[@aria-label="Go to next page"]')
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)
    except :
        ul = driver.find_element(By.CLASS_NAME, 'pagination_page_number')
        li_elements = ul.find_elements(By.TAG_NAME, 'li')
        last_page = int(li_elements[-1].text.strip())
        print(f"Last page: {last_page}")
        break

driver.quit()

Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Last page: 48


In [15]:
news_topics, news_contents = [], []
for num_page in tqdm(range(last_page)):
    page_link = f"https://www.innovestx.co.th/wealth-playlist/{num_page+1}"
    for topic, link in get_news(page_link):
        news_topics.append(topic.strip())
        news_contents.append(link)

100%|██████████| 48/48 [10:17<00:00, 12.86s/it]


In [18]:
import pandas as pd
data = {'News topics': news_topics, 'Links': news_contents}
df = pd.DataFrame(data)
df.to_csv('InnovestX_news.csv', index=False, encoding='utf-8-sig')